In [ ]:
pip install python-dotenv langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=9be9e263495f454469a3a3908d82a265b9fe694a5f85ac18ab1cda6a30e6a168
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
from langchain_groq import ChatGroq

groq_api = os.getenv("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")

In [ ]:
entity_types = ['person','school','award','company','product','characteristic', 'Location', 'organization', 'event', 'date', 'quantity','language', 'procedure', 'species']
relation_types = ['alumniOf','worksFor','hasAward','isProducedBy','hasCharacteristic','acquired','hasProject','isFounderOf', 'locatedIn', 'worksFor', 'bornIn', 'partOf', 'occured', 'causedBy', 'published']
prepositions = ['It', 'This', 'That', 'They', 'Those']
system_prompt = PromptTemplate(
    template = """
    You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
    Your task is to identify the entities and relations requested with the user prompt, from a given text.
    You must generate the output in a JSON containing a list with JSON objects having the following keys: "head", "head_type", "relation", "tail", and "tail_type".
    The "head" key must contain the text of the extracted entity with one of the types from the provided list in the user prompt.
    The "head_type" key must contain the type of the extracted head entity which must be one of the types from {entity_types} and do not include "prepositions" etc as entities.
    The "relation" key must contain the type of relation between the "head" and the "tail" which must be one of the relations from {relation_types} or similar.
    The "tail" key must represent the text of an extracted entity which is the tail of the relation, and the "tail_type" key must contain the type of the tail entity from {entity_types}.
    Attempt to extract as many entities and relations as you can.

    IMPORTANT NOTES:
    - Don't add any explanation and text.
    """,
    input_variables=["entity_types","relation_types"],
)


system_message_prompt = SystemMessagePromptTemplate(prompt = system_prompt)

examples = [
        {
            "text":"Adam is a software engineer in Microsoft since 2009, and last year he got an award as the Best Talent" ,
            "head": "Adam",
            "head_type": "person",
            "relation": "worksFor",
            "tail": "Microsoft",
            "tail_type": "company"
        },
        {
            "text":"Adam is a software engineer in Microsoft since 2009, and last year he got an award as the Best Talent" ,
            "head": "Adam",
            "head_type": "person",
            "relation": "hasAward",
            "tail": "Best Talent",
            "tail_type": "award"
        },
        {
            "text":"Microsoft is a tech company that provide several products such as Microsoft Word" ,
            "head": "Microsoft Word",
            "head_type": "product",
            "relation": "isproducedBy",
            "tail": "Microsoft",
            "tail_type": "company"
        },
        {
            "text":"Microsoft Word is a lightweight app that accessible offline" ,
            "head": "Microsoft Word",
            "head_type": "product",
            "relation": "hasCharacteristic",
            "tail": "lightweight app",
            "tail_type": "characteristic"
        },
        {
            "text":"Microsoft Word is a lightweight app that accessible offline" ,
            "head": "Microsoft Word",
            "head_type": "product",
            "relation": "hasCharacteristic",
            "tail": "accesible offline",
            "tail_type": "characteristic"
        },
    ]

class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Microsoft, Apple, John")
    head_type: str = Field(description="type of the extracted head entity like person, company, etc")
    relation: str = Field(description="relation between the head and the tail entities")
    tail: str = Field(description="extracted second or tail entity like Microsoft, Apple, John")
    tail_type: str = Field(description="type of the extracted tail entity like person, company, etc")

parser = JsonOutputParser(pydantic_object=ExtractedInfo)

human_prompt = PromptTemplate(
    template = """ Based on the following example, extract entities and relations from the provided text.\n\n

    Use the following entity types, don't use other entity that is not defined below:
    # ENTITY TYPES:
    {entity_types}

    Use the following relation types, don't use other relation that is not defined below:
    # RELATION TYPES:
    {relation_types}

    Below are a number of examples of text and their extracted entities and relationshhips.
    {examples}

    For the following text, generate extract entitites and relations as in the provided example.\n{format_instructions}\nText: {text}""",
    input_variables=["entity_types","relation_types","examples","text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


api = "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"
# model = ChatOllama(model = "mistral",temperature=0)
# model = ChatOllama(model = "llama3",temperature=0)
model = ChatGroq(temperature=0, model_name="llama3-70b-8192", api_key=api)
chain = LLMChain(llm=model, prompt=chat_prompt)

In [ ]:
pip install datasets faiss-cpu==1.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# SQUAD DATASET
from datasets import load_dataset
from tqdm import tqdm
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import os

# Load the SQuAD dataset

dataset = load_dataset("rajpurkar/squad")

# Prepare the dataset (for 20 rows)
def prepare_dataset(dataset):
    dataset = dataset.select(range(100))  # Select only the first 20 rows
    dataset = dataset.map(lambda x: {
        'user_input': x['question'],
        'reference': x['answers']['text'][0] if x['answers']['text'] else '',
        'response': '',  # Placeholder for the generated response
        'retrieved_contexts': [x['context']]  # Make sure this is a list of contexts
    })
    return dataset

# Prepare the dataset
prepared_dataset = prepare_dataset(dataset["train"])

# 2. **Initialize the Embedding Model**
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 3. **Create a FAISS Index from the Dataset's Contexts**
def create_faiss_index(dataset, embedding_model):
    contexts = [example['retrieved_contexts'][0] for example in dataset]
    context_embeddings = embedding_model.encode(contexts, convert_to_numpy=True)

    # Convert context embeddings to numpy array and create the FAISS index
    context_embeddings = context_embeddings.astype(np.float32)

    # Initialize FAISS index
    index = faiss.IndexFlatL2(context_embeddings.shape[1])  # L2 distance metric

    # Add embeddings to the index
    index.add(context_embeddings)

    return index

# Create FAISS index from the dataset
faiss_index = create_faiss_index(prepared_dataset, embedding_model)

# 4. **Retrieve Relevant Contexts from FAISS**
def retrieve_relevant_context(query, faiss_index, embedding_model, top_k=1):
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    query_embedding = query_embedding.astype(np.float32).reshape(1, -1)

    # Perform a similarity search (retrieving top_k nearest contexts)
    distances, indices = faiss_index.search(query_embedding, top_k)

    return indices

# Initialize distilbart model and tokenizer for local inference
distilbart_model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

# 5. **Generate Responses with Retrieved Context**
def generate_rag_responses(dataset, faiss_index, distilbart_model, tokenizer, embedding_model):
    responses = []
    for example in tqdm(dataset):
        user_input = example['user_input']

        # Retrieve the most relevant context(s)
        relevant_indices = retrieve_relevant_context(user_input, faiss_index, embedding_model)
        relevant_contexts = [dataset[int(idx)]['retrieved_contexts'][0] for idx in relevant_indices[0]]  # Convert idx to int

        # Combine the user question and retrieved context for distilbart input
        combined_input = f"Question: {user_input}\nContext: {'. '.join(relevant_contexts)}"

        # Encode the input and generate a response
        inputs = tokenizer(combined_input, return_tensors="pt", truncation=True)
        outputs = distilbart_model.generate(**inputs)

        # Decode the generated output
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        responses.append(response)

    return responses

# Generate responses and add them to the 'response' column
responses = generate_rag_responses(prepared_dataset, faiss_index, distilbart_model, tokenizer, embedding_model)

# Fill the 'response' column with generated responses
for idx, response in enumerate(responses):
    prepared_dataset[idx]['response'] = response

# Print the first row to check the response
print(prepared_dataset[0])  # Print the first row to verify


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

100%|██████████| 100/100 [14:48<00:00,  8.88s/it]

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}, 'user_input': 'To whom did the Virgin Mary allegedly a

In [ ]:
# DATASET PRE-PROCESSING

contexts = list()
questions = list()
answers = list()
count = 0

for entry in prepared_dataset:
  con = entry["context"]
  que = entry["question"]
  ans = entry["answers"]["text"]

# context already extracted
  if con in contexts:
    que_list = questions[count - 1]
    que_list.append(que)
    questions[count - 1] = que_list

    ans_list = answers[count - 1]
    ans_list.append(ans)
    answers[count - 1] = ans_list

# new context
  else:
    contexts.append(con)
    count += 1
    q_list = list()
    q_list.append(que)

    a_list = list()
    a_list.append(ans)

    questions.append(q_list)
    answers.append(a_list)


for i in range(len(contexts)):

  print(f"context {i+1} : {contexts[i]}")

  # for j in range(len(questions[i])):
  #   print(f"question {j+1} : {questions[i][j]}")
  #   print(f"answer : {answers[i][j]}")

  print("\n")

context 1 : Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


context 2 : As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic

In [ ]:
result = []
for cont in contexts:
  sentences = cont.split('. ')

  # Iterate over each sentence
  # print("Context:", i+1)
  for sentence in sentences:
      # Process each sentence
      response  = chain.run(entity_types = entity_types, relation_types = relation_types, examples = examples, text = sentence)

      try:
        response = eval(response)
        for res in response:
          if res is not None:
            relation = {}
            relation['sentence'] = sentence
            relation['head'] = res['head']
            relation['relation'] = res['relation']
            relation['tail'] = res['tail']
            result.append(relation)
      except:
          pass
      print(response)

[{'head': 'school', 'head_type': 'school', 'relation': 'hasCharacteristic', 'tail': 'Catholic character', 'tail_type': 'characteristic'}]
[{'head': 'Main Building', 'head_type': 'Location', 'relation': 'locatedIn', 'tail': 'gold dome', 'tail_type': 'Location'}, {'head': 'golden statue', 'head_type': 'product', 'relation': 'isProducedBy', 'tail': 'Atop', 'tail_type': 'Location'}, {'head': 'golden statue', 'head_type': 'product', 'relation': 'hasCharacteristic', 'tail': 'golden', 'tail_type': 'characteristic'}, {'head': 'golden statue', 'head_type': 'product', 'relation': 'hasCharacteristic', 'tail': 'statue of the Virgin Mary', 'tail_type': 'characteristic'}]
[{'head': 'Christ', 'head_type': 'person', 'relation': 'locatedIn', 'tail': 'Main Building', 'tail_type': 'Location'}]
[{'head': 'Main Building', 'head_type': 'Location', 'relation': 'locatedIn', 'tail': 'Basilica of the Sacred Heart', 'tail_type': 'Location'}]
[{'head': 'Grotto', 'head_type': 'Location', 'relation': 'locatedIn', '

In [ ]:
print(result)
print(len(result))

[{'sentence': 'Architecturally, the school has a Catholic character', 'head': 'school', 'relation': 'hasCharacteristic', 'tail': 'Catholic character'}, {'sentence': "Atop the Main Building's gold dome is a golden statue of the Virgin Mary", 'head': 'Main Building', 'relation': 'locatedIn', 'tail': 'gold dome'}, {'sentence': "Atop the Main Building's gold dome is a golden statue of the Virgin Mary", 'head': 'golden statue', 'relation': 'isProducedBy', 'tail': 'Atop'}, {'sentence': "Atop the Main Building's gold dome is a golden statue of the Virgin Mary", 'head': 'golden statue', 'relation': 'hasCharacteristic', 'tail': 'golden'}, {'sentence': "Atop the Main Building's gold dome is a golden statue of the Virgin Mary", 'head': 'golden statue', 'relation': 'hasCharacteristic', 'tail': 'statue of the Virgin Mary'}, {'sentence': 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes"', 'head': 'Christ', '

In [ ]:
import csv

# Function to export relations to a CSV file
def export_relations_to_csv(triples, output_file):
    # Define the CSV headers
    headers = ["Sentence", "Head", "Relation", "Tail"]

    # Open the CSV file for writing
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write the headers

        # Write each relation in the KB
        for relation in triples:
          if relation["head"] not in prepositions:
            writer.writerow([
                relation['sentence'],          # sentence
                relation['head'],              # Head
                relation['relation'],          # Relation
                relation['tail']               # Tail
            ])

    print(f"Relations successfully exported to {output_file}")


# Export to a CSV file
export_relations_to_csv(result, "relations_groq.csv")


Relations successfully exported to relations_groq.csv
